In [ ]:
import cudf
from elasticsearch_dsl import Search as S
from connect import connection
from connect import ElasticsearchIndex as es_idx
from elasticsearch import Elasticsearch as E
import pandas as pd
from elasticsearch_dsl import Q

In [ ]:
df = cudf.DataFrame()

In [ ]:
class find:
    def dslq():
        x = S(using=c)
        x = x.index(es_idx)
        x = x.query("match", type__keyword="Dionaea")

In [ ]:
xs = S().using(connection)
query = Q("multi_match", query='Dionaea', fields=['type.keyword'])
xs = xs.query(query)
xs = xs.source(["type", "timestamp", "src_ip", "dest_ip", "dest_port"])
xs = xs[1:1000]

In [ ]:
res = xs.execute()
for hit in res:
    #xdf = cudf.DataFrame()
    xdf = cudf.DataFrame({
        '@time': hit.timestamp,
        'dstip': hit.dest_ip,
        'srcip': hit.src_ip,
        'type': hit.type,
        'dstport': hit.dest_port
    })
    df = df.append(xdf)

In [ ]:
print(df)

In [ ]:
print(len(df))

In [ ]:
#for hit in res:
    #print(hit.to_dict)

In [ ]:
bx = {
  "size": 1,
  "sort": [
    {
      "@timestamp": {
        "order": "desc",
        "unmapped_type": "boolean"
      }
    }
  ],
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "@timestamp",
        "fixed_interval": "30s",
        "time_zone": "America/Los_Angeles",
        "min_doc_count": 1
      }
    }
  },
  "stored_fields": [
    "*"
  ],
  "script_fields": {},
  "docvalue_fields": [
    {
      "field": "@timestamp",
      "format": "date_time"
    },
    {
      "field": "end_time",
      "format": "date_time"
    },
    {
      "field": "flow.start",
      "format": "date_time"
    },
    {
      "field": "start_time",
      "format": "date_time"
    },
    {
      "field": "timestamp",
      "format": "date_time"
    },
    {
      "field": "tls.notafter",
      "format": "date_time"
    },
    {
      "field": "tls.notbefore",
      "format": "date_time"
    }
  ],
  "_source": {
    "excludes": []
  },
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "range": {
            "@timestamp": {
              "gte": "2020-10-24T23:01:45.478Z",
              "lte": "2020-10-24T23:16:45.478Z",
              "format": "strict_date_optional_time"
            }
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  },
  "highlight": {
    "pre_tags": [
      "@kibana-highlighted-field@"
    ],
    "post_tags": [
      "@/kibana-highlighted-field@"
    ],
    "fields": {
      "*": {}
    },
    "fragment_size": 2147483647
  }
}

In [ ]:
xres = connection.search(index='logstash-*', body=bx, size=1)

In [ ]:
#xres = xres.execute()
for key, val in xres["hits"].items():
    
    print(key, val)

In [ ]:
for hit in xres:
    print(hit)

In [ ]:
xx = xres["hits"].items()
print(xx)